#### Linjär regression på en variabel

Vi kommer här att titta på endast en av x-variablerna (_Geometric_), då vi tidigare konstaterat att det finns hög grad av beroende mellan de olika x-variablerna, och därför blir den förra modellen otillförlitlig.

Många av beräkningarna blir likadana som i första exemplet, och vi utför dem därför lite "fler åt gången".

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import LR

data_path = "Small-diameter-flow.csv"
df = pd.read_csv(data_path, index_col=0)

y = df["Flow"]
X = np.column_stack([np.ones(y.shape[0]), df['Geometric']])


Nu söker vi alltså ett linjärt samband på formen
$$
y=\beta_0 + \beta_1 [Geometric]
$$


In [4]:

m = LR.LinearRegression()
b = m.fit(X, y)
print(f"Vi får β0 = {b[0]:.7f}, β1 = {b[1]:.7f}")


Vi får β0 = 5.3323804, β1 = 3.2400047


Vi noterar speciellt att $\beta_1$, alltså lutningsparameterna för _Geometric_, blir 3.24, att jämföra med 3.61 från första försöket. (Det går förstås inte att jämföra rakt av, många andra faktorer spelar in, men kan ändå ge en hint om hur lika eller olika modellerna blir.)

In [7]:
d = m.dimension(b)
n = m.sample_size(y)
print(f"d = {d}, n = {n}")

SSE = m.SSE(y, X, b)
var = m.variance(SSE, n, d)
print(f"Variansen är {var:.7f}")

S = m.deviation(var)
Syy = m.Syy(n, y)
SSR = m.SSR(Syy, SSE)
print(f"S = {S:.7f}, Syy = {Syy:.7f}, SSR = {SSR:.7f}")

d = 1, n = 198
Variansen är 0.0182068
S = 0.1349327, Syy = 425.1441930, SSR = 421.5756511


Värdet på standardavvikelsen har samma dimension som x-variablerna, men eftersom dessa var logaritmerade, så blir även S dimensionslös. Det är dock ett slags mått på hur stora avvikelserna är från medlet, i medeltal.

SSR och Syy säger oss inte heller så mycket i nuläget, utan behövs mest för att räkna fram de viktiga statistikor vi behöver för att avgöra modellens relevans.

Nu ska vi beräkna F-värdet ("_F-statistic_"). Ett högt F-värde (långt från 1) innebär att det finns ett relevant samband mellan vårt X och y, alltså att vår modell i någon mening "makes sense".

In [44]:
F = m.Fstatistic(SSR, d, var)
print(f"F = {F}")


F = 22398.7658956827


Ett högt värde, så det är ingen tvekan om att det finns ett tydligt samband. Fortfarande finns dock en risk att sambandet är "skevt", då det kan påverkas av att X-variablerna inte är oberoende, vilket vi återkommer till.

Vi kan även beräkna $R^2$ (ibland kallad _Rsq_), som ger ett mått på hur stor andel av variansen i materialet som kan förklaras av vår modell:

In [45]:
Rsq = m.Rsq(SSR, Syy)
print(f"Rsq = {Rsq}")


Rsq = 0.9971212473210772


Vi fick alltså ett mycket högt värde på $R^2$, 0,997 motsvarar inte mindre än 3 standardavvikelser (3 $\sigma$) ! En mycket stor del av variansen kan alltså förklaras av vår modell.

Men - kanske är det så att vår modell är "för bra för att vara sann"? Om våra x-variabler inte är oberoende, kan det påverka modellen i stor utsträckning. Det kan både vara så att de "ger varandra draghjälp", eller motverkar varandra. Det kan också vara så att någon variabel är mer eller mindre betydelselös.

För att testa detta, gör vi ett s.k. _dependency check_, "beroende-test". Vi beräknar Pearson-r för varje kombination av två x-variabler, i vårt fall blir det alltså sammanlagt tre tester. Om vi får r-värden nära 1 eller -1, innebär det att det är ett tydligt samband. I samband med detta får vi även fram _p-värdet_, som är ett mått på sannolikheten att det inte skulle finnas något samband. Ett p-värde nära 0 gör att vi kan vara mycket säkra på att det _finns_ ett samband.

In [46]:
# Dependency check

print(f"kinematic / geometric: r = {m.Pearsonr(X, 1, 2)[0]:.7f}, p = {m.Pearsonr(X, 1, 2)[1]}")
print(f"kinematic / inertial: r = {m.Pearsonr(X, 1, 3)[0]:.7f}, p = {m.Pearsonr(X, 1, 3)[1]}")
print(f"geometric / inertial: r = {m.Pearsonr(X, 2, 3)[0]:.7f}, p = {m.Pearsonr(X, 2, 3)[1]}")

kinematic / geometric: r = 0.8631351, p = 4.5604633624399433e-60
kinematic / inertial: r = 0.9686708, p = 1.588545639896567e-120
geometric / inertial: r = 0.9183300, p = 7.951572627158216e-81


Vi har alltså "tyvärr" tydliga beroenden mellan alla tre x-variablerna, vilket gör att den modell vi räknat ut är näst intill värdelös! Det kan vara så att de olika variablerna påverkar varandra, eller så kan en av variablerna "följa med på köpet" när en annan ökar, och därmed vara ganska irrelevant på egen hand!

Vi ska titta lite på signifikansen hos de olika β-parametrarna, för att se om vi kan få någon ledtråd. Vi börjar med att beräkna varians/kovarians-matrisen:

In [47]:
# Beräkna varians/kovarians-matris
c = m.var_covar(X, var)
print(c)

[[ 0.1786122  -0.01993074 -0.00513088  0.01567684]
 [-0.01993074  0.00237237  0.00043044 -0.00168614]
 [-0.00513088  0.00043044  0.00108079 -0.0008474 ]
 [ 0.01567684 -0.00168614 -0.0008474   0.00154512]]


Vi kommer att göra ett tvåsidigt relevanstest, där vi kommer att välja den minsta av de två "svansarna" som vi får genom att titta på den kumulativa distributionsfunktionen, respektive "survival-funktionen". För att göra detta test vehövs värdena längs diagonalen i varians/kovariansmatrisen: 

In [48]:
# beräkna signifikans för de enskilda β-parametrarna
sig=[]
for a in range(d+1):
    sig.append(m.significance(a,b,c,S))
for a in range(d+1):
    print(f"Relevansen för β{a} är {m.relevance(sig, a, n, d)}")
    

Relevansen för β0 är 1.369442865878956e-146
Relevansen för β1 är 2.279977868333894e-236
Relevansen för β2 är 0.0
Relevansen för β3 är 1.9192830886558436e-242


Vi ser att vi har värden extremt nära 0 i samtliga fall, det är alltså i princip omöjligt att de inte är signifikanta. Men återigen, detta kan påverkas av deras inbördes beroende.

Vi vill även presentera ett konfidensintervall för de olika β-parametrarna, och vi vill välja en konfidensnivå på 95%. Därmed behöver vi börja med att beräkna lämpligt $t_{\alpha/2}$ på T-kurvan, alltså hur många standardavvikelser vi avviker.

In [49]:
print(f"t-värdet är {stats.t.ppf(1-0.05/2,n-d-1)}")
for a in range(d+1):
    ci=m.confidence_interval(n,d,var,c,a)
    print(f"Konfidensintervallet för β{a} på 95%-nivån är mellan {b[a]+ci} och {b[a]-ci}")


t-värdet är 1.972267532579456
Konfidensintervallet för β0 på 95%-nivån är mellan -2.554534655293283 och -2.5650516160478123
Konfidensintervallet för β1 på 95%-nivån är mellan 0.8693212249244441 och 0.8681091588053166
Konfidensintervallet för β2 på 95%-nivån är mellan 3.610827229208975 och 3.610009129744773
Konfidensintervallet för β3 på 95%-nivån är mellan -0.7531986370351633 och -0.754176809399355


$t_{\alpha/2}$-värdet på ca 1,97 motsvarar alltså ungefär 2 standardavvikelser, så vi kan ge ovanstående mått på β-parametrarna med 95% säkerhet.

Men, som vi tidigare konstaterat så är alla dessa siffror egentligen ganska ointressanta, då vi konstaterat att vår modell har en stor svaghet; de olika x-variablerna är i olika grad beroende av varandra. Vi bör därför testa andra modeller, vi skulle exempelvis kunna testa att låta en variabel vara $x_2\cdot x_3$, ellar att göra en regression på bara två av x-variablerna, eller någon annan kombination. Det blir dock litte tids- och utrymmeskrävande, så vi nöjer oss med att helt enkelt göra en regression y som funktion av en enda av x-variablerna. Vi väljer _Geometric_ eftersom vi har tjuvtittat på en _scatter matrix_, och sambandet mellan _Geometric_ och _Flow_ ser ut att vara det allra tydligaste.

Vi kommer alltså att göra de flesta av ovanstående beräkningar igen, men nu bara i _en_ x-variabel. Detta görs i filen _Linear_Regr_extra.ipynb_.